In [33]:
import pandas as pd
import pytesseract
from PIL import Image
import re
import matplotlib.pyplot as plt

In [2]:
DATASET = '../dataset/subset_train.csv'
df = pd.read_csv(DATASET)

In [3]:
df.head()

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [20]:
TRAIN_IMAGE_PATH = '../images/subset_train'

def ocr_image(image_path):
    image = Image.open(image_path)
    custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ%., '
    text = pytesseract.image_to_string(image, config=custom_config)
    pattern = r'(\d+[\.\,]?\d*\s?[a-zA-Z%]*)'
    matches = re.findall(pattern, text)
    return matches

image_path = '/home/spellsharp/Amazon_ML_Challenge/images/subset_train/711SATIDrmL.jpg'
result = ocr_image(image_path)
print(result)

['1300mAh', '100C', '14.8V', '60Plug', '169g', '1,', '1.38Inches', '1.26Inches']


In [28]:
def get_image_path(x):
    pattern = r'(?<=images/I/)(.*)'
    match = re.search(pattern, x)
    assert match
    return TRAIN_IMAGE_PATH + '/' + match.group(1)

df['image_path'] = df['image_link'].apply(lambda x: get_image_path(x))

In [29]:
df.head()

,image_link,group_id,entity_name,entity_value,image_path
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram,../images/subset_train/61I9XdN6OFL.jpg
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup,../images/subset_train/71gSRbyXmoL.jpg
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram,../images/subset_train/61BZ4zrjZXL.jpg
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram,../images/subset_train/612mrlqiI4L.jpg
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram,../images/subset_train/617Tl40LOXL.jpg


In [30]:
df['extracted_text'] = df['image_path'].apply(lambda x: ocr_image(x))

In [31]:
df.head()

,image_link,group_id,entity_name,entity_value,image_path,extracted_text
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram,../images/subset_train/61I9XdN6OFL.jpg,"[1\nINGREDIENTMENAGER, 100%NATURELs, 00%]"
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup,../images/subset_train/71gSRbyXmoL.jpg,"[4g, 4 LSS, 4 , 4 J]"
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram,../images/subset_train/61BZ4zrjZXL.jpg,"[1Tablet, 0.709gEachservingcontainsApprox, 200..."
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram,../images/subset_train/612mrlqiI4L.jpg,[4gbBseecenee]
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram,../images/subset_train/617Tl40LOXL.jpg,"[1 , 40 fe, 7\nEse]"


In [41]:
# def show_image(path):
#     image = plt.imread(path)
#     plt.imshow(image)
#     plt.show()
# show_image('../images/subset_train/612mrlqiI4L.jpg')

In [42]:
df['unit'] = df['entity_value'].str.extract(r'(\w+)$')
df['entity_value'] = df['entity_value'].str.replace(r'\s*\w+$', '')

In [45]:
df = df.drop(columns=['image_link'])

In [46]:
df.head(30)

,group_id,entity_name,entity_value,image_path,extracted_text,unit
0,748919,item_weight,500.0,../images/subset_train/61I9XdN6OFL.jpg,"[1\nINGREDIENTMENAGER, 100%NATURELs, 00%]",gram
1,916768,item_volume,1.0,../images/subset_train/71gSRbyXmoL.jpg,"[4g, 4 LSS, 4 , 4 J]",cup
2,459516,item_weight,0.709,../images/subset_train/61BZ4zrjZXL.jpg,"[1Tablet, 0.709gEachservingcontainsApprox, 200...",gram
3,459516,item_weight,0.709,../images/subset_train/612mrlqiI4L.jpg,[4gbBseecenee],gram
4,731432,item_weight,1400,../images/subset_train/617Tl40LOXL.jpg,"[1 , 40 fe, 7\nEse]",milligram
5,731432,item_weight,1400,../images/subset_train/61QsBSE7jgL.jpg,"[1400mg, 1400,, 365vscaul]",milligram
6,731432,item_weight,1400,../images/subset_train/81xsq6vf2qL.jpg,"[2vegancapsules, 1400mg, 1 seetheingredientsin...",milligram
7,731432,item_weight,1400,../images/subset_train/71DiLRHeZdL.jpg,"[1400ne, 365o, 8,\n, 1 , 5 OF]",milligram
8,731432,item_weight,1400,../images/subset_train/91Cma3RzseL.jpg,"[100%, 10 e, 4 S, 7s]",milligram
9,731432,item_weight,1400,../images/subset_train/71jBLhmTNlL.jpg,"[6\n, 10 Horbaach]",milligram


In [47]:
df.to_csv('processed_subset_train.csv', index=False)